## Hello, torch
Follows the basic torch demonstration of image classifier.

In [43]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from periodiclogger import PeriodicLogger
import random

logger = PeriodicLogger(100)
batch_size = 32
epochs = 1

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [44]:
data_train = datasets.FashionMNIST(root="data", train=True, download=True, transform=ToTensor())
data_test = datasets.FashionMNIST(root="data", train=False, download=True, transform=ToTensor())

train_dataloader = DataLoader(data_test, batch_size=batch_size)
test_dataloader = DataLoader(data_train, batch_size=batch_size)

In [45]:
class NeuralNetwork(nn.Module):
	def __init__(self):
		super().__init__()
		self.flatten = nn.Flatten()
		self.linear_relu_stack = nn.Sequential(
			nn.Linear(28*28, 512),
			nn.ReLU(),
			nn.Linear(512,512),
			nn.ReLU(),
			nn.Linear(512, 10)
		)

	def forward(self, x):
		x = self.flatten(x)
		logits = self.linear_relu_stack(x)
		return logits

In [46]:
model = NeuralNetwork().to(device)
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [47]:
## training
model.train()

for i in range(epochs):
	for (x,y) in test_dataloader:
		x,y = x.to(device), y.to(device)
		pred = model(x)
		loss = loss_fn(pred, y)
		logger.log(loss)

		# backprop
		optimizer.zero_grad()
		loss.backward()
		optimizer.step()

tensor(2.3116, grad_fn=<NllLossBackward0>)
tensor(1.0440, grad_fn=<NllLossBackward0>)
tensor(1.0431, grad_fn=<NllLossBackward0>)
tensor(0.5885, grad_fn=<NllLossBackward0>)
tensor(0.5118, grad_fn=<NllLossBackward0>)
tensor(0.4898, grad_fn=<NllLossBackward0>)
tensor(0.4748, grad_fn=<NllLossBackward0>)
tensor(0.4570, grad_fn=<NllLossBackward0>)
tensor(0.2457, grad_fn=<NllLossBackward0>)
tensor(0.4891, grad_fn=<NllLossBackward0>)
tensor(0.3171, grad_fn=<NllLossBackward0>)
tensor(0.6129, grad_fn=<NllLossBackward0>)
tensor(0.4838, grad_fn=<NllLossBackward0>)
tensor(0.5558, grad_fn=<NllLossBackward0>)
tensor(0.6510, grad_fn=<NllLossBackward0>)
tensor(0.2671, grad_fn=<NllLossBackward0>)
tensor(0.4899, grad_fn=<NllLossBackward0>)
tensor(0.5030, grad_fn=<NllLossBackward0>)
tensor(0.4494, grad_fn=<NllLossBackward0>)


In [48]:
## testing
model.eval()
with torch.no_grad():
	for x,y in test_dataloader:
		x,y = x.to(device), y.to(device)
		pred = model(x)
		logger.log(loss_fn(pred, y).item())

0.33772021532058716
0.18791058659553528
0.47007110714912415
0.5518968105316162
0.412311315536499
0.46446552872657776
0.6353407502174377
0.6330277323722839
0.2881247401237488
0.5271233320236206
0.20862144231796265
0.4804372191429138
0.25594621896743774
0.35763826966285706
0.4302162528038025
0.6601241827011108
0.42624515295028687
0.292507529258728
0.18743085861206055


In [50]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
for i in range(5):
    j = random.randint(0,10000)
    x, y = data_test[j]
    with torch.no_grad():
        pred = model(x)
        best_index = pred[0].argmax(0)
        predicted, actual = classes[best_index], classes[y]
        prob = torch.softmax(pred[0], dim=0)[best_index]*100
        # logit = pred[0][best_index]
        # prob = (1/(1+torch.exp(-logit)))*100
        print(f'Predicted: {predicted}, Actual: {actual}, Confidence: {prob:.2f}%')

Predicted: Bag, Actual: Bag, Confidence: 98.57%
Predicted: Trouser, Actual: Trouser, Confidence: 99.96%
Predicted: Coat, Actual: Coat, Confidence: 92.86%
Predicted: Sneaker, Actual: Sneaker, Confidence: 99.73%
Predicted: T-shirt/top, Actual: Shirt, Confidence: 48.78%
